# Data generation

Esta es una forma de generar un dataset sintético con GPT-4 y una prompt.

El dataset debe seguir el formato indicado por OpenAI (para GPT-3.5 Turbo), pues vamos a hacer fine-tuning a un modelos de ellos:

* https://openai.com/blog/gpt-3-5-turbo-fine-tuning-and-api-updates
* https://platform.openai.com/docs/guides/fine-tuning

Es necesario instalar:

* pip install pandas
* pip install openai
* pip install openai tenacity

Utilicé la version 0.27.0 de openai (tenacity me daba problemas con la última versión).

In [4]:
import os
import openai
import random
from tenacity import retry, stop_after_attempt, wait_exponential

In [7]:
openai.api_key = "..." # Aquí debes colocar tu propia API key de OpenAI.

Escribe tu **prompt** (debe ser muy descriptiva).

Elige un valor para **temperature** (entre 0 y 1) que deseas usar al generar el dataset. Valores más bajos son ideales para tareas precisas, como escribir código, mientras que valores más altos son mejores para tareas creativas, como escribir historias.

Elige un valor para **number_of_examples**. Cuantos más generes, más tiempo tomará y será más costosa la generación del dataset. En general, más ejemplos conducirán a un modelo de mayor calidad. Por lo general, se recomienda un mínimo de 100 para comenzar.

In [2]:
prompt = "A model that takes in a puzzle-like reasoning-heavy question in English, and responds with a well-reasoned, step-by-step thought out response in Spanish."
temperature = 0.8
number_of_examples = 50


En el repositorio ya están creados los ejemplos de entrenamiento para esta prueba (**training_example.jsonl**).

Correr para generar el dataset. Esto gasta muchos créditos con GPT-4, cuidado.

* https://openai.com/pricing

In [ ]:
N_RETRIES = 3

@retry(stop = stop_after_attempt(N_RETRIES), wait = wait_exponential(multiplier = 1, min = 4, max = 70))

def generate_example(prompt, prev_examples, temperature = 0.5):
    messages = [
        {
            "role": "system",
            "content": f"You are generating data which will be used to train a machine learning model.\n\nYou will be given a high-level description of the model we want to train, and from that, you will generate data samples, each with a prompt/response pair.\n\nYou will do so in this format:\n```\nprompt\n-----------\n$prompt_goes_here\n-----------\n\nresponse\n-----------\n$response_goes_here\n-----------\n```\n\nOnly one prompt/response pair should be generated per turn.\n\nFor each turn, make the example slightly more complex than the last, while ensuring diversity.\n\nMake sure your samples are unique and diverse, yet high-quality and complex enough to train a well-performing model.\n\nHere is the type of model we want to train:\n`{prompt}`"
        }
    ]

    if len(prev_examples) > 0:
        if len(prev_examples) > 8:
            prev_examples = random.sample(prev_examples, 8)
        for example in prev_examples:
            messages.append({
                "role": "assistant",
                "content": example
            })

    response = openai.ChatCompletion.create(
        model = "gpt-4",
        messages = messages,
        temperature = temperature,
        max_tokens = 1000,
    )

    return response.choices[0].message["content"]

# Generar ejemplos:
prev_examples = []
for i in range(number_of_examples):
    print(f"Generating example {i}")
    example = generate_example(prompt, prev_examples, temperature)
    prev_examples.append(example)

print(prev_examples)

Tambien necesitamos generar un mensaje del sistema.

In [ ]:
def generate_system_message(prompt):
  response = openai.ChatCompletion.create(
      model = "gpt-4",
      messages = [
        {
          "role": "system",
          "content": "You will be given a high-level description of the model we are training, and from that, you will generate a simple system prompt for that model to use. Remember, you are not generating the system message for data generation -- you are generating the system message to use for inference. A good format to follow is `Given $INPUT_DATA, you will $WHAT_THE_MODEL_SHOULD_DO.`.\n\nMake it as concise as possible. Include nothing but the system prompt in your response.\n\nFor example, never write: `\"$SYSTEM_PROMPT_HERE\"`.\n\nIt should be like: `$SYSTEM_PROMPT_HERE`."
        },
        {
          "role": "user",
          "content": prompt.strip(),
        }
      ],
      temperature = temperature,
      max_tokens = 500,
  )

  return response.choices[0].message["content"]

system_message = generate_system_message(prompt)

print(f"The system message is: `{system_message}` Feel free to re-run this cell if you want a better result.")

Ahora, se colocan los ejemplos en un DataFrame.

In [ ]:
import json
import pandas as pd

# Crear listas vacias para almacenar prompts y las respuestas:
prompts = []
responses = []

# Analizar prompts y las respuestas de los ejemplos:
for example in prev_examples:
    try:
        split_example = example.split("-----------")
        prompts.append(split_example[1].strip())
        responses.append(split_example[3].strip())
    except:
        pass

# Crear un DataFrame.
df = pd.DataFrame({
    "prompt": prompts,
    "response": responses
})

# Para guardar el DataFrame en un archivo CSV:
# df.to_csv('training_examples.csv', index = False)

# Remover duplicados:
df = df.drop_duplicates()

print("There are " + str(len(df)) + " successfully-generated examples.")

# Crear una lista vacía para almacenar ejemplos de entrenamiento:
training_examples = []

# Crea ejemplos de entrenamiento en el formato necesario para el fine-tuning de GPT-3.5 Turbo:
for index, row in df.iterrows():
    training_example = {
        "messages": [
            {"role": "system", "content": system_message.strip()},
            {"role": "user", "content": row['prompt']},
            {"role": "assistant", "content": row['response']}
        ]
    }
    training_examples.append(training_example)

# Guardar ejemplos de entrenamiento en un archivo JSONL:
with open("training_examples.jsonl", "w") as f:
    for example in training_examples:
        f.write(json.dumps(example) + '\n')